In [4]:
from ultralytics import YOLO
import json
import pandas as pd
from yolo_config import Config
import os

## Agregate Folds

In [6]:
dataset = 'InBreast_Yolov8'


keys = ['metrics/precision(B)', 'metrics/recall(B)', 'metrics/mAP50(B)', 'metrics/mAP50-95(B)', 'fitness',
        'TP', 'FP', 'FN', 'TN']

for model_size in Config.MODEL_SIZES:
    for conf_threshold in Config.THRESHOLDS:
        aggregated_results = {k:0 for k in keys}

        model_dir = f'{model_size}_conf={conf_threshold}'
        model_path = os.path.join(Config.RUNS_DIR, Config.DATASET, model_size, model_dir)

        for fold in range(1,11):
            fold_dir = f'fold_{fold}'
            results_path = os.path.join(model_path, fold_dir, 'val', 'results.json')
            results = json.load(open(results_path))
            for k in keys:
                aggregated_results[k] += results[k]
            
        aggregated_results = {k:v/10 for k,v in aggregated_results.items()}
        aggregated_results_path = os.path.join(model_path, 'results.json')

        with open(aggregated_results_path, 'w+') as file:
            json.dump(aggregated_results, file)

        print(aggregated_results)
        
        # break # Threshold
    break # Model Size

{'metrics/precision(B)': 0.8731413559911483, 'metrics/recall(B)': 0.9276301258335632, 'metrics/mAP50(B)': 0.9268554533244533, 'metrics/mAP50-95(B)': 0.7406831321185215, 'fitness': 0.7593003642391148, 'TP': 9.5, 'FP': 0.9, 'FN': 0.6, 'TN': 0.0}
{'metrics/precision(B)': 0.8733084026251425, 'metrics/recall(B)': 0.9286497598542471, 'metrics/mAP50(B)': 0.9259188650891591, 'metrics/mAP50-95(B)': 0.7434004238519243, 'fitness': 0.7616522679756477, 'TP': 9.5, 'FP': 0.9, 'FN': 0.6, 'TN': 0.0}
{'metrics/precision(B)': 0.8714902208069608, 'metrics/recall(B)': 0.9175386487431361, 'metrics/mAP50(B)': 0.9231288379398379, 'metrics/mAP50-95(B)': 0.7411285222627374, 'fitness': 0.7593285538304474, 'TP': 9.5, 'FP': 0.6, 'FN': 0.6, 'TN': 0.0}
{'metrics/precision(B)': 0.8925377585324256, 'metrics/recall(B)': 0.8902909937417747, 'metrics/mAP50(B)': 0.9124316733127982, 'metrics/mAP50-95(B)': 0.734347829139194, 'fitness': 0.7521562135565546, 'TP': 9.2, 'FP': 0.3, 'FN': 0.9, 'TN': 0.0}


## Summarize Results

In [9]:
all_results = []
model_names = []

for model_size in Config.MODEL_SIZES:
    for conf_threshold in Config.THRESHOLDS:
        model_name = f'{model_size}_conf={conf_threshold}'
        aggregated_results_path = os.path.join(Config.RUNS_DIR, Config.DATASET, 
                                               model_path, 'results.json')

        all_results.append(json.load(open(aggregated_results_path)))
        model_names.append(model_name)

    #     break # Threshold
    break # Model Size

all_results_path = os.path.join(Config.RUNS_DIR, Config.DATASET, 'all_results.csv')
all_results = pd.DataFrame(all_results, index=model_names)
all_results.to_csv(all_results_path)

In [10]:
all_results

,metrics/precision(B),metrics/recall(B),metrics/mAP50(B),metrics/mAP50-95(B),fitness,TP,FP,FN,TN
yolov8n.pt_conf_0.1,0.873141,0.927630,0.926855,0.740683,0.759300,9.5,0.9,0.6,0.0
yolov8n.pt_conf_0.2,0.873308,0.928650,0.925919,0.743400,0.761652,9.5,0.9,0.6,0.0
yolov8n.pt_conf_0.3,0.871490,0.917539,0.923129,0.741129,0.759329,9.5,0.6,0.6,0.0
yolov8n.pt_conf_0.4,0.892538,0.890291,0.912432,0.734348,0.752156,9.2,0.3,0.9,0.0
